# Notebook for Creating High Wage Outputs

#### This notebook is currently written to create high wage outputs for 2020.

In [1]:
import pandas as pd
import numpy as np
import string
import warnings
import os
import re
from jqi_functions import *
warnings.filterwarnings('ignore')

## Creating IPUMS dataframe

#### IPUMS Data
`cleaned_ipums` is a function to generate a cleaned pandas dataframe using IPUMS data, filtering it down to California only and the desired year. The year needs to be entered in string format as a parameter.

In [2]:
ca_ipums = cleaned_ipums('2020')

In [3]:
county_info = pd.read_csv('data/county_to_regions_key.csv')

#### Cost of living needs to be updated each year.

In this case, the 2020 United Way Real Cost Measure has not been published, so I will continue using the data from 2019.

In [4]:
cost_of_living = pd.read_csv('data/cost_of_living/united-way-col-1A1PS1C2019.csv')

In [5]:
ca_ipums = pd.merge(ca_ipums, county_info, on = 'COUNTYFIP')

Cleaning `ca_ipums` after merging with `county_info` and `cost_of_living`.

In [6]:
ca_ipums = ca_ipums[['INDNAICS', 'PERWT', 'INCWAGE',
       'NAICS Code', 'Industry Title_x', 'Main_Code', 'Sub_1_Code', 'Sub_2_Code', 'Sub_3_Code', 'Sub_4_Code', 'County', 'Rural/Urban', 'CDI Regions']]

In [7]:
ca_ipums = pd.merge(ca_ipums, cost_of_living, left_on = 'CDI Regions', right_on = 'Regions')
ca_ipums = ca_ipums.rename(columns = {'Cost of Living':'Regional COL'})

In [8]:
ca_ipums = pd.merge(ca_ipums, cost_of_living, left_on = 'Rural/Urban', right_on = 'Regions')
ca_ipums = ca_ipums.rename(columns = {'Cost of Living':'Rural/Urban COL'})

In [9]:
ca_ipums = pd.merge(ca_ipums, cost_of_living, left_on = 'County', right_on = 'Regions')
ca_ipums = ca_ipums.rename(columns = {'Cost of Living':'County COL'})

In [10]:
ca_ipums = ca_ipums[['INDNAICS', 'PERWT', 'INCWAGE',
       'NAICS Code', 'Industry Title_x', 'Main_Code', 'Sub_1_Code', 'Sub_2_Code', 'Sub_3_Code', 'Sub_4_Code', 'County', 'Rural/Urban', 'CDI Regions',
                    'Regional COL', 'Rural/Urban COL', 'County COL']]

In [11]:
ca_ipums['Regional Rural/Urban'] = ca_ipums['CDI Regions'] + ' ' + ca_ipums['Rural/Urban']

In [12]:
ca_ipums = pd.merge(ca_ipums, cost_of_living, left_on = 'Regional Rural/Urban', right_on = 'Regions')
ca_ipums = ca_ipums.rename(columns = {'Cost of Living':'Regional Rural/Urban COL'})

In [13]:
ca_ipums = ca_ipums.rename(columns = {'Industry Title_x':'Industry Title'})
ca_ipums = ca_ipums[['INDNAICS', 'PERWT', 'INCWAGE',
       'NAICS Code', 'Industry Title', 'Main_Code', 'Sub_1_Code', 'Sub_2_Code', 'Sub_3_Code',
       'Sub_4_Code', 'County', 'Rural/Urban', 'CDI Regions', 'Regional Rural/Urban',
                    'Regional COL', 'Rural/Urban COL', 'County COL', 'Regional Rural/Urban COL']]

In [14]:
ca_ipums['Industry Title'] = normalize_titles(ca_ipums['Industry Title'])

View of final `ca_ipums` dataframe.

In [15]:
ca_ipums.head()

,INDNAICS,PERWT,INCWAGE,NAICS Code,Industry Title,Main_Code,Sub_1_Code,Sub_2_Code,Sub_3_Code,Sub_4_Code,County,Rural/Urban,CDI Regions,Regional Rural/Urban,Regional COL,Rural/Urban COL,County COL,Regional Rural/Urban COL
0,928110p4,79.0,62000,928110p4,us marines,900,910,919,919,919,San Diego,Urban,San Diego-Imperial,San Diego-Imperial Urban,51652,79472,79472,79472
1,928110p4,64.0,24000,928110p4,us marines,900,910,919,919,919,San Diego,Urban,San Diego-Imperial,San Diego-Imperial Urban,51652,79472,79472,79472
2,928110p4,66.0,30000,928110p4,us marines,900,910,919,919,919,San Diego,Urban,San Diego-Imperial,San Diego-Imperial Urban,51652,79472,79472,79472
3,928110p4,58.0,24000,928110p4,us marines,900,910,919,919,919,San Diego,Urban,San Diego-Imperial,San Diego-Imperial Urban,51652,79472,79472,79472
4,928110p4,75.0,20000,928110p4,us marines,900,910,919,919,919,San Diego,Urban,San Diego-Imperial,San Diego-Imperial Urban,51652,79472,79472,79472


## Create county lookup dataframe

Expanding the `county_info` dataframe to include cost of living metrics. This dataframe is used when industry information in a geographic area is too sparse and the next largest geographic area needs to be used instead.

In [16]:
county_info = county_info[['County', 'Rural/Urban', 'CDI Regions']]

In [17]:
county_info['Regional Rural/Urban'] = county_info['CDI Regions'] + ' ' + county_info['Rural/Urban']

In [18]:
county_info = pd.merge(county_info, cost_of_living, left_on = 'County', right_on = 'Regions')

In [19]:
county_info = county_info.rename(columns = {'Cost of Living':'County COL'})
county_info = county_info.drop(columns=['Regions'])

In [20]:
county_info = pd.merge(county_info, cost_of_living, left_on = 'Regional Rural/Urban', right_on = 'Regions')

In [21]:
county_info = county_info.rename(columns = {'Cost of Living':'Regional Rural/Urban COL'})
county_info = county_info.drop(columns=['Regions'])

In [22]:
county_info = pd.merge(county_info, cost_of_living, left_on = 'CDI Regions', right_on = 'Regions')

In [23]:
county_info = county_info.rename(columns = {'Cost of Living':'Regional COL'})
county_info = county_info.drop(columns=['Regions'])

In [24]:
county_info = pd.merge(county_info, cost_of_living, left_on = 'Rural/Urban', right_on = 'Regions')

In [25]:
county_info = county_info.rename(columns = {'Cost of Living':'Rural/Urban COL'})
county_info = county_info.drop(columns=['Regions'])

In [26]:
county_info['State COL'] = cost_of_living.iloc[11][1]

View of final `county_info` dataframe.

In [27]:
county_info.head()

,County,Rural/Urban,CDI Regions,Regional Rural/Urban,County COL,Regional Rural/Urban COL,Regional COL,Rural/Urban COL,State COL
0,Alameda,Urban,Bay Area,Bay Area Urban,88296,94329,93392,79472,74448
1,Contra Costa,Urban,Bay Area,Bay Area Urban,86284,94329,93392,79472,74448
2,Solano,Urban,Bay Area,Bay Area Urban,66751,94329,93392,79472,74448
3,San Mateo,Urban,Bay Area,Bay Area Urban,112606,94329,93392,79472,74448
4,Santa Clara,Urban,Bay Area,Bay Area Urban,107879,94329,93392,79472,74448


## Create EDD Dataframe

#### EDD Data
The year for EDD data must be specified.

In [34]:
# edd = pd.read_csv('data/edd/edd_2020_parsed.csv')
edd = pd.read_csv('data/edd/cut_edd_2020.csv')

In [30]:
# edd = clean_edd(edd)

View of final `edd` dataframe.

In [35]:
edd.head()

,Area Name,Date,Industry Title,Current Employment
0,Alameda,12/01/2020,Health Care and Social Assistance,109400
1,Alameda,12/01/2020,Federal Government,8800
2,Alameda,12/01/2020,Local Government,71900
3,Alameda,12/01/2020,Real Estate and Rental and Leasing,9800
4,Alameda,12/01/2020,"Professional, Scientific and Technical S",76100


## Load NAICS Crosswalk

In [37]:
naics = pd.read_csv('data/ipums/naics_parsed_crosswalk.csv').drop_duplicates(subset='INDNAICS').reset_index().iloc[:,1:]

In [38]:
naics['Industry Title'] = normalize_titles(naics['Industry Title'])
naics['Sub_1_Code'] = [str(x) for x in naics['Sub_1_Code']]
naics['Main_Code'] = [str(x) for x in naics['Main_Code']]

View of final `naics` dataframe.

In [99]:
naics.head()

,Industry Title,INDNAICS,Main_Code,Sub_1_Code,Sub_2_Code,Sub_3_Code,Sub_4_Code
0,crop production,111,111,111,111,111,111
1,support activities for agriculture and forestry,115,111,111,111,111,111
2,animal production and aquaculture,112,111,111,111,111,111
3,fishing hunting and trapping,114,111,111,111,111,111
4,forestry except logging,113m,100,113,113,113,113


## Add High Wage Features

`add_geo_high_wages` is a function that adds the following engineered features:
- Above Threshold (Number of records above respective cost of living threshold)
- Weighted above threshold (Above Threshold multiplied by person weight variable)
- Unweighted industry counts (Number of records in that industry)
- Weighted industry counts (Sum of person weight values in that industry)
- Weighted high wage percentage (Weighted Above Threshold divided by Weighted Industry Counts as a percentage)

The features are created for the following geographical levels:
- County
- Regional Rural/Urban
- Region
- Rural/Urban
- California

In [40]:
ca_ipums_hw = add_geo_high_wages(ca_ipums)

In [41]:
ca_ipums_hw['Sub_1_Code'] = [str(x) for x in ca_ipums_hw['Sub_1_Code']]
ca_ipums_hw['Main_Code'] = [str(x) for x in ca_ipums_hw['Main_Code']]

View of final `ca_ipums_hw` dataframe.

In [100]:
ca_ipums_hw.head().T

,0,1,2,3,4
INDNAICS,928110p4,928110p4,928110p4,928110p4,928110p4
PERWT,79.0,64.0,66.0,58.0,75.0
INCWAGE,62000,24000,30000,24000,20000
NAICS Code,928110p4,928110p4,928110p4,928110p4,928110p4
Industry Title,us marines,us marines,us marines,us marines,us marines
Main_Code,900,900,900,900,900
Sub_1_Code,910,910,910,910,910
Sub_2_Code,919,919,919,919,919
Sub_3_Code,919,919,919,919,919
Sub_4_Code,919,919,919,919,919


## Create High Wage Outputs Dataframe

`edd_to_hw` is the function that outputs the values needed to create the high wage output dataframe. This portion of the notebook runs through every unique combination of county, industry, and date, to get that respective output and add it to the dataframe.

Getting unique values for each county, industry (as a parsed code), and date.

In [43]:
counties_edd = edd['Area Name'].unique()

In [44]:
parsed_codes = set(list(edd['Main_Code'].unique()) + list(edd['Sub_1_Code'].unique()) + list(edd['Sub_2_Code'].unique()) + list(edd['Sub_3_Code'].unique()) + list(edd['Sub_4_Code'].unique()))

In [45]:
dates_edd = edd['Date'].unique()

Initializing empty lists for the function's outputs to later be joined in a dataframe.

In [101]:
industries = []
dates = []
counties = []
counts = []
emp_counts = []

In [102]:
total_iterations = len(counties_edd) * len(parsed_codes) * len(dates_edd)

For loop to populate lists for the high wage outputs. This will take some time to finish running.

In [103]:
progress_count = 0
for county in counties_edd:
    for code in parsed_codes:
        for date in dates_edd:
            output, hw, industry, emp_count = edd_to_hw(edd, ca_ipums_hw, naics, county_info, county, str(code), date, 10)
            industries.append(industry)
            dates.append(date)
            counties.append(county)
            counts.append(hw)
            emp_counts.append(emp_count)
            progress_count += 1
            if progress_count % 10440 == 0:
                percent_done = int((progress_count / total_iterations) * 100)
                print(f'Progress: {percent_done}% Complete')

Progress: 10% Complete
Progress: 20% Complete
Progress: 30% Complete
Progress: 40% Complete
Progress: 50% Complete
Progress: 60% Complete
Progress: 70% Complete
Progress: 80% Complete
Progress: 90% Complete
Progress: 100% Complete


Creating a cleaned dataframe from the output lists.

In [104]:
df_dict = {'Industry':industries, 'Date':dates, 'County':counties, 'High Wage Count':counts, 'Employment Count':emp_counts}
hw_output = pd.DataFrame(df_dict)
hw_output = hw_output[hw_output['Industry'].notna()]
hw_output['Date']= pd.to_datetime(hw_output['Date'])
hw_output['High Wage Count'] = hw_output['High Wage Count'].astype(int)
hw_output = hw_output.sort_values(by=['Industry', 'County', 'Date'])
hw_output = pd.merge(hw_output, cost_of_living, left_on='County', right_on='Regions')
hw_output = hw_output[['Industry', 'Date', 'County', 'High Wage Count', 'Employment Count', 'Cost of Living']]
hw_output = hw_output.drop_duplicates()

View of final `hw_output` dataframe.

In [105]:
hw_output.head()

,Industry,Date,County,High Wage Count,Employment Count,Cost of Living
0,accounting tax preparation bookkeeping and pay...,2020-01-01,Los Angeles,13745,45200.0,80216
1,accounting tax preparation bookkeeping and pay...,2020-01-01,Los Angeles,13745,45200.0,80216
2,accounting tax preparation bookkeeping and pay...,2020-02-01,Los Angeles,15265,50200.0,80216
3,accounting tax preparation bookkeeping and pay...,2020-02-01,Los Angeles,15265,50200.0,80216
4,accounting tax preparation bookkeeping and pay...,2020-03-01,Los Angeles,15204,50000.0,80216


Code to export the dataframe as a CSV file - change file path if needed and uncomment to run.

In [70]:
# hw_output.to_csv('data/hw_outputs_2020.csv', encoding='utf-8', index=False)